In [1]:
import mne
import numpy as np
%matplotlib qt5

In [23]:
sfreq = 250
info = mne.create_info([f'CH{i}' for i in range(4)], sfreq=sfreq, ch_types=['eeg' for i in range(4)])
times = np.linspace(0, 3, 250, endpoint=False)
sine = np.sin(20*np.pi * times)
sine1 = np.sin(15*np.pi * (times - np.pi/2))
sine2 = np.sin(15*np.pi * (times - np.pi))
sine3 = np.sin(15*np.pi * (times - 3*np.pi/2)) + np.random.rand(len(times))
data = np.array([sine, sine1, sine2, sine3])
raw = mne.io.RawArray(data, info)

Creating RawArray with float64 data, n_channels=4, n_times=250
    Range : 0 ... 249 =      0.000 ...     0.996 secs
Ready.


In [24]:
epochs = mne.make_fixed_length_epochs(raw, duration=0.25, preload=False)

Not setting metadata
Not setting metadata
4 matching events found
No baseline correction applied
0 projection items activated


In [25]:
epochs.load_data()
csd = mne.time_frequency.csd_fourier(epochs, 1, 100)

Loading data for 4 events and 63 original time points ...
0 bad epochs dropped
Computing cross-spectral density from epochs...


C:\Users\seand\AppData\Local\Temp/ipykernel_4352/374947002.py:2: RuntimeWarning: Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
  csd = mne.time_frequency.csd_fourier(epochs, 1, 100)
100%|██████████| CSD epoch blocks : 4/4 [00:00<00:00,  266.60it/s]

[done]


In [26]:
mat = csd.sum(19.8, 24).get_data()
mat

array([[ 0.00192476+0.j        , -0.00643035-0.0006452j ,
        -0.00188014+0.00618104j,  0.00542598+0.00320665j],
       [-0.00643035+0.0006452j ,  0.11660999+0.j        ,
         0.02255   -0.11440883j, -0.10859696-0.04483678j],
       [-0.00188014-0.00618104j,  0.02255   +0.11440883j,
         0.11660999+0.j        ,  0.02298977-0.11521768j],
       [ 0.00542598-0.00320665j, -0.10859696+0.04483678j,
         0.02298977+0.11521768j,  0.11897451+0.j        ]])

In [27]:
def coherence(csd, i, j):
    return np.absolute(np.absolute(csd[i][j]) ** 2 / (csd[i][i] * csd[j][j]))

In [28]:
coherence_matrix = [[coherence(mat, i, j) for j in range(4)] for i in range(4)]
coherence_matrix

[[1.0, 0.1860834558630182, 0.18596980101015145, 0.17346895608287874],
 [0.1860834558630182, 1.0, 0.9999995972784177, 0.9949559266299253],
 [0.18596980101015145,
  0.9999995972784177,
  0.9999999999999999,
  0.9949563793026579],
 [0.17346895608287874,
  0.9949559266299253,
  0.9949563793026579,
  0.9999999999999999]]

In [29]:
cov = mne.compute_covariance(epochs)
cov

Computing rank from data with rank=None
    Using tolerance 1.5e-08 (2.2e-16 eps * 4 dim * 1.7e+07  max singular value)
    Estimated rank (eeg): 4
    EEG: rank 4 computed from 4 data channels with 0 projectors
Reducing data rank from 4 -> 4
Estimating covariance using EMPIRICAL
Done.
Number of samples used : 252
[done]


C:\Users\seand\AppData\Local\Temp/ipykernel_4352/3237834791.py:1: RuntimeWarning: Epochs are not baseline corrected, covariance matrix may be inaccurate
  cov = mne.compute_covariance(epochs)


<Covariance | size : 4 x 4, n_samples : 251, data : [[ 0.49908777  0.02224244  0.01059424 -0.02344159]
 [ 0.02224244  0.50199203  0.09707504 -0.46000296]
 [ 0.01059424  0.09707504  0.50199203  0.07675952]
 [-0.02344159 -0.46000296  0.07675952  0.80524887]]>